In [94]:
import random

In [95]:
num_students = 450
num_universities = 8
min_num_groups = 1
min_group_size = 20
max_group_size = 30
max_num_groups = 5    

students = list(range(num_students))
universities = list(range(num_universities))

In [96]:
# Генерация меток
student_labels = {
    'is_target': [0] * num_students,
    'is_contract': [0] * num_students,
    'debts': [],
    'rating': []}

# Генерация меток целевых студентов
num_target_students = int(0.05 * num_students)
target_students = random.sample(students, num_target_students)

for target in target_students:
    student_labels['is_target'][target] = 1

# Генерация меток контрактников
num_contract_students = int(0.3 * num_students)
contract_students = random.sample(students, num_contract_students)

for contract in contract_students:
    student_labels['is_contract'][contract] = 1

# Генерация академических задолженностей
def generate_debts():
    return random.choices(range(8), weights=[10, 5, 3, 3, 2, 2, 1, 1], k=1)[0]

student_labels['debts'] = [generate_debts() for _ in students]

# Генерация баллов рейтинга
student_labels['rating'] = [random.randint(0, 500) for _ in students]

In [97]:
for i in range(num_students):
    print(f"Студент {i}: Целевой - {student_labels['is_target'][i]}, "
          f"Контрактник - {student_labels['is_contract'][i]}, "
          f"Долги - {student_labels['debts'][i]}, "
          f"БРС - {student_labels['rating'][i]}")

Студент 0: Целевой - 0, Контрактник - 0, Долги - 1, БРС - 392
Студент 1: Целевой - 0, Контрактник - 0, Долги - 1, БРС - 127
Студент 2: Целевой - 0, Контрактник - 0, Долги - 2, БРС - 416
Студент 3: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 103
Студент 4: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 227
Студент 5: Целевой - 0, Контрактник - 0, Долги - 3, БРС - 22
Студент 6: Целевой - 0, Контрактник - 0, Долги - 5, БРС - 229
Студент 7: Целевой - 0, Контрактник - 1, Долги - 0, БРС - 0
Студент 8: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 93
Студент 9: Целевой - 0, Контрактник - 0, Долги - 2, БРС - 105
Студент 10: Целевой - 0, Контрактник - 1, Долги - 0, БРС - 267
Студент 11: Целевой - 0, Контрактник - 0, Долги - 1, БРС - 34
Студент 12: Целевой - 0, Контрактник - 0, Долги - 4, БРС - 55
Студент 13: Целевой - 0, Контрактник - 0, Долги - 7, БРС - 255
Студент 14: Целевой - 0, Контрактник - 1, Долги - 4, БРС - 30
Студент 15: Целевой - 0, Контрактник - 0, Долги - 5, БРС - 151
Студент 1

In [98]:
# Генерация предпочтений студентов
preferences_s = {}
for student in students:
    if student_labels['debts'][student] > 5:
        preferences_s[student] = []  # Равнодушие к университетам, пустой список
    else:
        preferences_s[student] = random.sample(universities, k=num_universities)  # Выбор университетов

# Генерация предпочтений университетов
preferences_u = {}
for uni in universities:
    target_students_sorted = sorted(
        [s for s in students if student_labels['is_target'][s] == 1],
        key=lambda s: -student_labels['rating'][s]
    )

    other_students = sorted(
        [s for s in students if student_labels['is_target'][s] == 0 and student_labels['debts'][s] <= 5],
        key=lambda s: (-student_labels['rating'][s], -student_labels['is_contract'][s])
    )

    debt_students = sorted(
        [s for s in students if student_labels['debts'][s] > 5],
        key=lambda s: -student_labels['rating'][s]
    )

    # Формируем полные предпочтения для университета
    preferences_u[uni] = target_students_sorted + other_students + debt_students


# Сформируем квоты для университетов
quotas = [max_group_size * max_num_groups for _ in range(num_universities)]

In [99]:
# Подсчет студентов, которые поставили университет первым приоритетом
priority_count = {uni: 0 for uni in universities}
target_priority_count = {uni: 0 for uni in universities}

for student in students:
    if preferences_s[student]:  # Проверяем, есть ли предпочтения
        first_preference = preferences_s[student][0]
        priority_count[first_preference] += 1  # Увеличиваем счетчик для данного университета
        # Учитываем только целевых студентов, которые поставили этот университет первым
        if student_labels['is_target'][student] == 1 and first_preference == preferences_s[student][0]:
            target_priority_count[first_preference] += 1  # Увеличиваем счетчик целевых студентов

# Определение обязательных университетов
mandatory_universities = [
    uni for uni in universities 
    if target_priority_count[uni] > 4 or priority_count[uni] > 60
]

# Количество обязательных университетов
num_mandatory_universities = len(mandatory_universities)

# Объем резерва
reserve_size = num_mandatory_universities * min_num_groups * min_students_in_group
reserve_students = []

# Список студентов без предпочтений
students_without_preferences = []
for student in students:
    if not preferences_s[student]:
        students_without_preferences.append(student)

# Список оставшихся нецелевых студентов
remaining_students = []
for student in students:
    if student_labels['is_target'][student] == 0:
        remaining_students.append(student)
        
# Заполнение резерва студентами без предпочтений
students_without_preferences_sorted = sorted(
    students_without_preferences, 
    key=lambda s: student_labels['rating'][s]
)

for student in reversed(students_without_preferences_sorted):
    if len(reserve_students) < reserve_size:
        reserve_students.append(student)
    else:
        break

# Заполнение резерва оставшимися нецелевыми студентами
remaining_students_sorted = sorted(
    remaining_students,
    key=lambda s: student_labels['rating'][s]
)

for student in reversed(remaining_students_sorted):
    if len(reserve_students) < reserve_size:
        reserve_students.append(student)
    else:
        break

# Вывод результатов
for uni in universities:
    print(f"Университет {uni}: Студенты, поставившие первым приоритетом: {priority_count[uni]}, "
          f"целевиков: {target_priority_count[uni]}")

print(f"Размер резерва студентов: {len(reserve_students)}")
print(f"Резерв студентов: {reserve_students}")

Университет 0: Студенты, поставившие первым приоритетом: 48, целевиков: 2
Университет 1: Студенты, поставившие первым приоритетом: 44, целевиков: 3
Университет 2: Студенты, поставившие первым приоритетом: 52, целевиков: 1
Университет 3: Студенты, поставившие первым приоритетом: 64, целевиков: 4
Университет 4: Студенты, поставившие первым приоритетом: 51, целевиков: 4
Университет 5: Студенты, поставившие первым приоритетом: 46, целевиков: 2
Университет 6: Студенты, поставившие первым приоритетом: 54, целевиков: 3
Университет 7: Студенты, поставившие первым приоритетом: 55, целевиков: 2
Размер резерва студентов: 20
Резерв студентов: [51, 305, 45, 429, 326, 245, 246, 244, 368, 26, 442, 231, 220, 171, 73, 193, 13, 227, 119, 40]


In [100]:
# Реализация первого шага распределения
university_matches = {uni: [] for uni in range(num_universities)}  # Сопоставления университетов
student_matches = {student: None for student in range(num_students)}  # Сопоставления студентов

# Распределение целевых студентов по университетам
for student in students:
    if student_labels['is_target'][student] == 1:  # Проверяем, является ли студент целевым
        if preferences_s[student]:  # Проверяем, есть ли предпочтения
            first_preference = preferences_s[student][0]  # Получаем первый приоритет
            university_matches[first_preference].append(student)  # Добавляем студента в университет
            student_matches[student] = first_preference  # Помечаем, в какой университет распределен студент

# Создание списка оставшихся студентов
remaining_students_after_distribution = []
for student in students:
    if student_matches[student] is None:
        remaining_students_after_distribution.append(student)

# Вывод результатов
for uni, matched_students in university_matches.items():
    print(f"Университет {uni}: Студенты - {matched_students}")

Университет 0: Студенты - [329, 449]
Университет 1: Студенты - [295, 310, 330]
Университет 2: Студенты - [195]
Университет 3: Студенты - [140, 317, 339, 398]
Университет 4: Студенты - [156, 158, 343, 387]
Университет 5: Студенты - [129, 211]
Университет 6: Студенты - [62, 133, 320]
Университет 7: Студенты - [48, 315]


In [101]:
len(remaining_students_after_distribution)

429

In [102]:
mandatory_universities

[3]

In [103]:
def gale_shapley(num_students, num_universities, quotas, preferences_s, preferences_u, university_matches, student_matches, mandatory_universities, reserve_students):
    # Инициализация
    reserve_set = set(reserve_students)  # Студенты в резерве
    # Убираем студентов из резерва из remaining_students_after_distribution
    free_students = [s for s in remaining_students_after_distribution if s not in reserve_set]  
    proposed = {student: 0 for student in range(num_students)}  # Счетчик предложений

    while free_students:
        # Вытаскиваем свободного студента
        student = free_students[0]
        student_pref = preferences_s[student]

        # Находим наиболее предпочтительный университет, которому студент еще не делал предложение
        uni_index = proposed[student]

        if uni_index < len(student_pref):  # Здесь используем длину предпочтений, чтобы не сломаться на студентах с пустыми списками
            uni = student_pref[uni_index]
            proposed[student] += 1

              # Если университет свободен, совмещаем
            if len(university_matches[uni]) < quotas[uni]:
                university_matches[uni].append(student)
                university_matches[uni].sort(key=preferences_u[uni].index)
                student_matches[student] = uni
                free_students.pop(0)
                
                # Проверяем, связан ли университет с обязательным профилем и обрабатываем резерв
                if uni in mandatory_universities and reserve_students:
                    student_from_reserve = reserve_students.pop(0)  # Берем первого студента из резерва
                    free_students.append(student_from_reserve)  # Добавляем студента в список свободных
            else:
                # Если университет занят, проверяем, хочет ли он заменить текущего худшего студента
                current_student = university_matches[uni][-1]
                uni_pref = preferences_u[uni]

                if uni_pref.index(student) < uni_pref.index(current_student):
                    university_matches[uni].pop()
                    university_matches[uni].append(student)
                    university_matches[uni].sort(key=uni_pref.index)
                    student_matches[student] = uni
                    student_matches[current_student] = None
                    free_students.pop(0)
                    free_students.append(current_student)
                    
                    # Проверяем, связан ли университет с обязательным профилем и обрабатываем резерв
                    if uni in mandatory_universities and reserve_students:
                        student_from_reserve = reserve_students.pop(0)  # Берем первого студента из резерва
                        free_students.append(student_from_reserve)  # Добавляем студента во свободные списки
        else:
            # Если студент сделал все предложения, он становится несопоставленным
            free_students.pop(0)

    # Список студентов с пустыми предпочтениями
    students_with_no_preferences = [s for s in range(num_students) if len(preferences_s[s]) == 0]
    # Основной цикл распределения завершен

    # Теперь распределяем студентов с пустыми предпочтениями случайным образом
    for student in students_with_no_preferences:
        available_universities = [uni for uni in range(num_universities) if len(university_matches[uni]) < quotas[uni]]
        if available_universities:  # Если есть доступные университеты
            uni = random.choice(available_universities)  # Случайный университет
            university_matches[uni].append(student)  # Записываем студента в университет
            student_matches[student] = uni  # Обновляем сопоставление студента
    
    # Список несопоставленных студентов
    unmatched_students = [s for s in range(num_students) if student_matches[s] is None]

    return university_matches, student_matches, unmatched_students

In [104]:
university_matches, student_matches, unmatched_students = gale_shapley(
    num_students, num_universities, quotas, preferences_s, preferences_u, university_matches, student_matches, mandatory_universities, reserve_students
)

In [105]:
for uni, matched_students in university_matches.items():
    print(f"Профиль {uni}: Студенты - {matched_students}")
    print(f"Всего на профиле {uni} {len(matched_students)} студентов")
print("Незачисленные студенты:")
print(unmatched_students)

Профиль 0: Студенты - [329, 449, 179, 230, 146, 376, 37, 207, 69, 29, 89, 101, 410, 222, 77, 96, 150, 324, 401, 252, 284, 404, 106, 145, 103, 66, 319, 375, 214, 283, 217, 268, 104, 197, 24, 388, 253, 113, 430, 199, 355, 125, 118, 204, 405, 403, 154, 44, 193, 251, 366, 422, 447]
Всего на профиле 0 53 студентов
Профиль 1: Студенты - [295, 330, 310, 225, 420, 286, 209, 138, 196, 402, 391, 357, 412, 116, 249, 139, 370, 309, 361, 277, 36, 389, 347, 50, 288, 428, 162, 90, 130, 381, 15, 97, 271, 232, 60, 202, 308, 39, 208, 350, 108, 298, 75, 282, 13, 51, 227, 305, 326, 368, 424]
Всего на профиле 1 51 студентов
Профиль 2: Студенты - [195, 341, 127, 261, 431, 164, 337, 151, 190, 168, 378, 400, 256, 328, 383, 274, 267, 83, 157, 353, 87, 174, 132, 335, 243, 78, 367, 240, 203, 32, 255, 169, 348, 440, 4, 399, 416, 443, 299, 316, 215, 374, 413, 74, 307, 131, 53, 218, 354, 114, 59, 314, 73, 246, 429]
Всего на профиле 2 55 студентов
Профиль 3: Студенты - [317, 398, 339, 140, 419, 379, 318, 110, 134, 3